In [5]:
import dotenv
dotenv.load_dotenv()
import openai
import os

In [6]:
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

In [7]:
import pandas as pd
import evaluate
from transformers import pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_openai import ChatOpenAI
import os
import getpass
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from langchain.llms import HuggingFaceHub

/usr/local/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
query = 'Can I double major?'

In [35]:
context = """
"Double Major or Major/Minor Programs
Every UCSC undergraduate is required to complete a major program. Some students choose to pursue a double major, or a major/minor program. Pursuing a double major or major/minor program is a highly individual decision, and should be based on a student's educational and career goals. For many students, pursuing a single major and including enrichment such as: internships, research opportunities, or more classes than are required in their major discipline is just as challenging and rewarding as a double major. Talk with faculty in your major, your academic advisors, and Career Coaches about the benefits and drawbacks of pursuing more than a single major.

To declare a double major or major/minor program, declare your first major (if it's not already declared) through the process listed here. Once your first major is approved and declared, add your second major or minor using the process on the same page. Be sure to let your advisors know that you're planning a double major or major/ minor program! You must file a quarter-by-quarter academic plan showing that you are able to complete the program within the maximum number of quarters of enrollment available to you at UCSC. Students who enter as frosh are expected to complete their degree requirements in four years of enrollment (and are allowed a maximum of five years of enrollment), and students who enter as junior transfer students are expected to complete their degree requirements in two years of enrollment (and are allowed a maximum of three years of enrollment). See the Catalog for more information about limits on the number of enrollment terms and credits allowed.

Major and Minor Requirements
To qualify for a bachelor’s degree at UCSC, you must complete the minimum requirements for a major program, as well as satisfy university, campus, and college requirements.
At UCSC, you have the option of pursuing a single major, a double major, or a combined major. The minimum requirements for an established major program are set by the sponsoring department. (If you are a transfer student, the department will determine which of your transferable courses may be used to satisfy major requirements.) The major involves substantial work in the discipline and requires no fewer than 40 upper-division or graduate credits. Only courses in which you earn a grade of Pass, C (2.0 GPA), or better satisfy major or minor requirements.
Additional Majors or Minors
To complete multiple majors and minors, you must fulfill all of the requirements for all majors and minors declared, including the comprehensive requirement for each major. In general, a single thesis may not be used for more than one major. You may count courses for more than one major or minor, as long as each major includes 40 upper-division credits not used to satisfy the minimum upper-division credits of any other major or minor, and each minor includes at least 25 upper-division credits not used to satisfy the minimum upper-division credits of any other major or minor.
The diploma of a student who has completed a double major in history and music, for example, would read “Bachelor of Arts with Majors in History and Music.”
Declaring a Major
The field of interest you indicate on your application to the University of California, Santa Cruz, does not automatically place you in a major. You are advised to declare your major as soon as possible. Students who enter as lower-division students are required to declare a major before enrolling in the equivalent of their third year.* You will not be allowed to enroll in classes for the equivalent of your third year until you have declared a major. Junior transfer students must declare a major during their second quarter at UC Santa Cruz by the deadline printed in the Academic and Administrative Calendar.
Be sure you are aware of all the necessary criteria for qualifying for the major. Petition to declare a major as soon as you complete major qualification requirements, or reach your declaration deadline quarter, whichever comes first..
Determine the requirements for possible majors as soon as possible. Certain majors require substantial preparation, with many interlocking course sequences and qualifying grades in major foundation courses. If you intend to pursue such a major, start work toward it early in your undergraduate career, and review your progress toward qualification regularly. (Review majors that interest you in the Academic Programs section of the catalog). Academic advisors can offer assistance in selecting courses appropriate to your individual needs.
*Note: This is the year you would become a junior given normal progress to degree. For example, if you transfer to UCSC as a beginning sophomore, it is your second year here.
Comprehensive Requirement
Every major at UCSC includes a senior exit requirement designed to integrate the knowledge and skills learned throughout the curriculum. This capstone requirement may be a senior thesis, senior seminar, comprehensive examination, or some other integrative experience. Choices for satisfying this requirement are specified with the requirements for each major.
Minor Programs
See Fields of Study for undergraduate minors currently offered at UC Santa Cruz. Completion of a minor is optional. If you wish, you may complete more than one minor.
The sponsoring department establishes the course requirements for a minor. The minor involves substantial work in the discipline and requires no fewer than 25 upper-division or graduate credits. The minor appears on your official transcript but not on your diploma.
Combined Major
A combined major allows you to complete a course of study involving two disciplines offered as regular programs at UC Santa Cruz.
Examples of combined majors include environmental studies/economics and Latin American and Latino studies/politics. A combined major is designed by faculty representatives from both disciplines. In general, fewer courses are required than for a double major, and students complete the comprehensive requirements as specified for each combined major. Combined majors currently available are listed in the footnotes in the Fields of Study.
The diploma of a student who has completed a combined major in environmental studies and economics, for example, would read “Bachelor of Arts with a Major in Environmental Studies/Economics.”
Individual Major
Academically strong students have the option of developing an individual major. Individual majors must be vetted and approved by the Committee on Educational Policy (CEP), and must be distinct from any approved major program or possible combination of major programs. Regular deadlines to declare the major also apply to individual majors; students should declare their individual major by the end of their sophomore year.
Because the process of developing and proposing an individual major requires rigorous planning, and approval is not guaranteed, students are advised to pursue an existing major plan until a proposal has been submitted and approved by CEP. Make an appointment with your college academic advisor or preceptor as soon as possible if you are considering this option.
Forming an individual major can be challenging, since three faculty members in the discipline(s) must agree to serve on a committee to supervise the major, and the major must be approved by the Committee on Educational Policy. Guidelines, policies, and forms for individual majors can be found on the Academic Senate website: Individual Major Guidelines and Information, and in Academic Senate Regulation 10.4.5.
Individual majors must include at least 10 courses (50 credits), of which at least eight (40 credits) must be upper-division. No more than two of the required courses may be independent study courses. Students who cannot meet their original study plan for the major must consult with the committee chair, and in some cases, the Committee on Educational Policy, for approval of any changes. All students who complete an individual major are awarded a bachelor of arts.
Catalog Rights
Effective for all undergraduates who entered in fall quarter 1993 or after, students may follow the degree requirements from either the University of California, Santa Cruz General Catalog published at the time of entering UC Santa Cruz or subsequent catalog(s). Students need not follow a catalog in its entirety but may elect to follow different catalog years for their college requirements, university and general education requirements, requirements of their major(s), and requirements of any minor(s).
Catalog year will initially be set for the first year of enrollment at UCSC. Students may elect to follow requirements from other catalog year(s) when petitioning to declare their major, or by talking with their advisors. All requirements for graduation outlined in the catalog(s) selected must be met before graduation. Changing catalog year(s) is done by contacting your major advisor (for major or minor requirements) or your college advisor (for college, university, and general education requirements).
Students transferring from other collegiate institutions may elect to meet as graduation requirements one of the following:
those in effect at the time of transfer to UCSC;
those subsequently established; or
those in effect when the student entered a previous collegiate institution, provided that entry was not more than three years prior to the time of transfer to UCSC.
Students who seek readmission to UC Santa Cruz after a break in attendance greater than two years (six fall, winter, or spring quarters) must adhere to the graduation requirements in effect at the time of readmission or those subsequently established.
Although general education requirements are determined by catalog year, the specific courses that satisfy a particular GE requirement can change from year to year. The Office of the Registrar annually publishes an updated list of courses that satisfy each GE requirement (except Disciplinary Communication). Since the GE status of a course may change, to fulfill a GE requirement, a student must take the course in a year in which its GE status is recognized.
The courses that satisfy the Disciplinary Communication GE requirement for each major are published in each year’s General Catalog. Students should consult the General Catalog for the year in which they plan to take their DC course(s) to find out the courses that are needed.
Students who entered prior to 1993 should see an advisor. Their catalog year(s) for graduation, whether the year they entered UCSC or subsequent year(s), will be decided at the discretion of their major department and/or their college.
The policy and guidelines for catalog rights are overseen by the Academic Senate Committee on Educational Policy. To read the full policy text, please see CEP: Catalog Rights.
Institutional Responsibility
Undergraduate students who have made significant progress toward a degree in a specific major can assume that a degree will be granted if they meet all catalog degree requirements and maintain continuous enrollment and progress.
Should UCSC find it necessary to discontinue a specific major, every effort will be made to allow currently enrolled majors to complete their degrees within a reasonable period of time. This may include (1) movement to a similar or related degree track; (2) substitution of requirements; (3) development of an individual major proposal; or (4) completion of courses at another University of California campus through the Intercampus Visitor Program. Students with questions concerning this policy should contact their major and college advising offices.
In all cases, any financial obligations are the responsibility of the individual student involved.

Since transfer students are expected to have completed most (if not all) of the lower-division coursework required for admission to their major, a change of major prior to admission will not be possible. Admitted students have the option to change their proposed major using the ""Update Your Major"" link available in your MyUCSC portal. Please note that only those majors available to you will be displayed."
"""

In [36]:
response = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "user", "content": f"""
        Given this query and context, return the query in concise first order logic (FOL). 
        
        Example: 
        Query: What classes does the boating center offer to community members?, 
        Context: "UC Santa Cruz Community Boating Center is a community-based boating program that connects students and community members to the Monterey Bay through recreational activities including sailing, rowing and kayaking.
        UC Santa Cruz Community Boating Center staff and veteran participants provide a familiarity with the equipment, skills, and water safety that comes from a lifetime of enjoying these sports.
        The Community Boating Center also connects people who are interested in these sports to one another, whether you are a beginner seeking to learn something new or a experienced sailor, rower, and/or kayaker looking to improve your skills and get time on the water. Whether you've never been on a boat before, or want to build on old skills, we have a class for you! From 2-week junior summer camps to quarterly physical education classes for UC Santa Cruz Students, you'll be cruising around Monterey Bay with ease.

        Our club exists to give community members, UC Santa Cruz students, faculty, and staff access to equipment that makes it possible to enjoy our beautiful Monterey Bay. The club provides members a unique opportunity to use any number of sailing and rowing vessels under the weekend supervision of the Boating Center dockmaster.
        Our boats:
        Sailing vessels for weekend use currently include RS Quests, Lasers, RS Visions, a SC27, a Santana 22, a J 24, an Olson 30, and an O'Day 34.
        The rowing fleet exists of eight Edon training shells, two MAAS AEROS 20s, two MAAS 24s, Bay 21, Pegasus, WinTech 30' double, and a custom-built wooden double. 
        Want to take a boat out? Make sure to pay your membership dues and boat fees online ahead of time. The boating center will no longer be accepting cash payments on-site.
        Sailing Hours:
        Friday 4:00-7:00 PM (during daylight savings)
        Saturday 12:00-5:00 PM
        Sunday 12:00-5:00 PM
        Members of the Community Boating Club have access to sailboats and rowing shells during club hours. Members must be approved for boat use by the boating directors before checking out a rowing shall or sailboat. Check our calendar for the next available checkout day. If you do not have prior sailing or rowing experience, we offer classes to build your confidence on the water! For class information, click here.
        Sailors and rowers are evaluated based on experience and allowed boat use accordingly.

        Boating Club Hours
        The Community Boating Center is open 2 days a week.
        Saturday: 12:00 PM - 5:00 PM
        Sunday: 12:00 PM - 5:00 PM
        .

        FOL: ClassesOfferedByBoating Club = Sailing ∧ Rowing ∧ Kayaking
        
        Prompt: 
        Query: {query}, 
        Context: {context}
        .
        
        FOL:
        """}
    ]
)
fol = response.choices[0].message.content
print(fol)

∀s (UCSCUndergrad(s) → (MayDoubleMajor(s) ↔ FilesPlanWithinMaxEnrollmentQuarters(s) ∧ ∀m (Declared(s,m) → CompletesRequirements(s,m) ∧ AtLeast40UniqueUpperDivCredits(s,m))))


In [37]:
response = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {
            "role": "user",
            "content": f"""
        
        Convert this first order logic query to a concise prolog database.  
                       
        Example:
        FOL: 
        ∀x (CanBeMemberOfBoatingClub(x) ↔ CommunityMember(x) ∨ UCSCStudent(x) ∨ UCSCFaculty(x) ∨ UCSCStaff(x))
        
        Prolog:
        % Facts
        community_member(community_member).
        ucsc_student(ucsc_student).
        ucsc_faculty(ucsc_faculty).
        ucsc_staff(ucsc_staff).

        % Rule encoding the equivalence
        can_be_member_of_boating_club(X) :-
            community_member(X);
            ucsc_student(X);
            ucsc_faculty(X);
            ucsc_staff(X).
        
        Prompt:
        FOL: 
        {fol}.
        
        Prolog:
        [output]
        
        Omit ```prolog ``` and any intro.
        """,
        }
    ],
)
prolog = response.choices[0].message.content
print(prolog)

% Facts
ucsc_undergrad(ucsc_undergrad).
files_plan_within_max_enrollment_quarters(files_plan_within_max_enrollment_quarters).
declared(declared_s, declared_m).
completes_requirements(completes_requirements_s, completes_requirements_m).
at_least_40_unique_upper_div_credits(at_least_40_unique_upper_div_credits_s, at_least_40_unique_upper_div_credits_m).

% Rule
may_double_major(S) :-
    ucsc_undergrad(S),
    files_plan_within_max_enrollment_quarters(S),
    forall(declared(S,M),
           (completes_requirements(S,M),
            at_least_40_unique_upper_div_credits(S,M))).


In [38]:
with open("rules.pl", "w") as f:
    f.write(prolog)

In [39]:
response = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "user", "content": f"""
        Here I have a prolog rule and query, return the prolog query to run.
        Store all valid results in a variable X. 
        
        Example:
        Query: Who can be a member of the Boating Club?,
        Rulebase:
        % Facts
        community_member(community_member).
        ucsc_student(ucsc_student).
        ucsc_faculty(ucsc_faculty).
        ucsc_staff(ucsc_staff).

        % Rule encoding the equivalence
        can_be_member_of_boating_club(X) :-
            community_member(X);
            ucsc_student(X);
            ucsc_faculty(X);
            ucsc_staff(X).
        
        Prolog Query:
        findall(Y, can_be_member_of_boating_club(Y), X), write(X), nl
        
        Query: 
        {query}, 
        Rulebase: 
        {prolog} 
        Only output the prolog query without an intro or '?- '. Format the query as such: Query, write(X), nl
        """}
    ]
)
prolog_query = response.choices[0].message.content
print(prolog_query)

findall(Y, may_double_major(Y), X), write(X), nl


In [40]:
import subprocess

# File containing Prolog code
prolog_file = "rules.pl"
pl_query = prolog_query #[3:-1]

# This assumes your .pl file defines a predicate called `query/0`
result = subprocess.run(
    ["swipl", "-s", prolog_file, "-g", pl_query, "-t", "halt"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print("STDOUT:")
print(result.stdout)
print("STDERR:")
print(result.stderr)


STDOUT:
[]

STDERR:



In [41]:
response = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {
                "role": "user",
                "content": f"Here is the prolog output and error pipes from a given query, if there was a prolog error, say there was an error, else, answer the query in natural language. Original query: {query}, prolog query: {prolog_query}, stdout: {result.stdout}, stderr: {result.stderr}",
            }
        ],
    )
response.choices[0].message.content

'No. Based on the results (no eligible cases found), the system indicates you cannot double major.'

In [ ]:
# from pyswip import Prolog

# prolog = Prolog()
# prolog.consult("rules.pl")

# # Run a query
# results = list(prolog.query(c_query))

# # Print the results
# for result in results:
#     print(result)
